In [29]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

# 문서 로드(Load Documents)

In [30]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [31]:
# from langchain_community.document_loaders import DirectoryLoader
# 
# loader = DirectoryLoader(".", glob="data/*.pdf", show_progress=True)
# docs = loader.load()

In [32]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/개인정보 보호법(법률)(제19234호)(20240315).pdf")

# 페이지 별 문서 로드
docs = loader.load_and_split()

In [33]:
print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[2500:2600])

문서의 수: 41

[메타데이터]

{'source': 'data/개인정보 보호법(법률)(제19234호)(20240315).pdf', 'page': 0}

========= [앞부분] 미리보기 =========



# 문서 분할(Split Documents)

In [34]:
# langchain 패키지에서 RecursiveCharacterTextSplitter 클래스를 가져옵니다.
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    # 정말 작은 청크 크기를 설정합니다.
    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

In [35]:
splits = recursive_text_splitter.split_documents(docs)
len(splits)

193

# 임베딩(Embedding) & 벡터DB(VectorStore)
* 임베딩
    - 문서를 벡터 표현으로 변환
* 벡터DB
    - 변환된 벡터를 DB에 저장

In [36]:
# from langchain_community.embeddings import FastEmbedEmbeddings
# 
# embeddings_model = FastEmbedEmbeddings()

In [37]:
from langchain_community.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [38]:
# from langchain_community.embeddings import HuggingFaceEmbeddings
# 
# embeddings_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

In [39]:
from langchain_community.vectorstores import Chroma

# Chroma DB 적용
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)

# Retriever
* 쿼리가 주어지면 문서를 반환하는 인터페이스

In [40]:
retriever = vectorstore.as_retriever()

In [41]:
# query = "공공데이터법의 목적을 요약해줘"
# search_result = retriever.get_relevant_documents(query)
# print(search_result)

[Document(page_content='우에는 다음 각 호의 내용이 포함된 문서로 하여야 한다. <개정 2023. 3. 14.>\n1. 위탁업무 수행 목적 외 개인정보의 처리 금지에 관한 사항\n2. 개인정보의 기술적ㆍ관리적 보호조치에 관한 사항\n3. 그 밖에 개인정보의 안전한 관리를 위하여 대통령령으로 정한 사항', metadata={'page': 12, 'source': 'data/개인정보 보호법(법률)(제19234호)(20240315).pdf'}), Document(page_content='우에는 다음 각 호의 내용이 포함된 문서로 하여야 한다. <개정 2023. 3. 14.>\n1. 위탁업무 수행 목적 외 개인정보의 처리 금지에 관한 사항\n2. 개인정보의 기술적ㆍ관리적 보호조치에 관한 사항\n3. 그 밖에 개인정보의 안전한 관리를 위하여 대통령령으로 정한 사항', metadata={'page': 12, 'source': 'data/개인정보 보호법(법률)(제19234호)(20240315).pdf'}), Document(page_content='14.>\n1. 제8조의2에 따른 개인정보 침해요인 평가에 관한 사항\n2. 제9조에 따른 기본계획 및 제10조에 따른 시행계획에 관한 사항\n3. 개인정보 보호와 관련된 정책, 제도 및 법령의 개선에 관한 사항\n4. 개인정보의 처리에 관한 공공기관 간의 의견조정에 관한 사항\n5. 개인정보 보호에 관한 법령의 해석ㆍ운용에 관한 사항\n6. 제18조제2항제5호에 따른 개인정보의 이용ㆍ제공에 관한 사항', metadata={'page': 3, 'source': 'data/개인정보 보호법(법률)(제19234호)(20240315).pdf'}), Document(page_content='14.>\n1. 제8조의2에 따른 개인정보 침해요인 평가에 관한 사항\n2. 제9조에 따른 기본계획 및 제10조에 따른 시행계획에 관한 사항\n3. 개인정보 보호와 관련된 정책, 제도 및 법령의 개선에 관한 사항\n4. 개인정보의 

# 프롬프트
* 프롬프트 엔지니어링
    - 주어진 데이터(context)를 토대로 우리가 원하는 결과를 도출할 때 중요한 역할을 함

In [42]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

# 언어모델 생성(Create LLM)

In [43]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")

In [44]:
def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)

In [45]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [52]:
# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = ""
response = rag_chain.invoke(question)

In [53]:
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

[HUMAN]
이 문서는 개인정보 보호에 대한 문서인 5줄로 요약해줘.

[AI]
이 문서는 개인정보 보호법에 따라 위탁업무 수행 목적 외 개인정보 처리 금지, 기술적ㆍ관리적 보호조치, 대통령령으로 정한 안전한 관리 사항을 포함하고 있다.
